In [46]:
import pandas as pd
import numpy as  np
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


In [42]:
rawdata = pd.read_csv("data/raw/LifeExpectancyData.csv")
# Options for missing features - dropna(), drop(), fillna()
# drop the missing values from our target value
data = rawdata.dropna(axis=0, subset=['Life expectancy'])
data.isnull().sum()

Country                              0
Year                                 0
Status                               0
Life expectancy                      0
Adult Mortality                      0
infant deaths                        0
Alcohol                            193
percentage expenditure               0
Hepatitis B                        553
Measles                              0
BMI                                 32
under-five deaths                    0
Polio                               19
Total expenditure                  226
Diphtheria                          19
HIV/AIDS                             0
GDP                                443
Population                         644
thinness  1-19 years                32
thinness 5-9 years                  32
Income composition of resources    160
Schooling                          160
dtype: int64

In [57]:
# Now we can impute missing values for numerical attributes with fillna() 
# compute the median/mean value on the data and use that to fill the missing values 
# We are also missing 21% of observations for "population" so figure out how to handle this later

# NOTE: Cannot do this with categoricals, so drop them first. 
num_only_df = data.drop(["Country", "Status"], axis=1)
imputer = SimpleImputer(strategy="mean")
X = imputer.fit_transform(num_only_df)
imputed_df = pd.DataFrame(X, columns=num_only_df.columns)  # Now we can put this back into a DataFrame


Year                               0
Life expectancy                    0
Adult Mortality                    0
infant deaths                      0
Alcohol                            0
percentage expenditure             0
Hepatitis B                        0
Measles                            0
BMI                                0
under-five deaths                  0
Polio                              0
Total expenditure                  0
Diphtheria                         0
HIV/AIDS                           0
GDP                                0
Population                         0
thinness  1-19 years               0
thinness 5-9 years                 0
Income composition of resources    0
Schooling                          0
dtype: int64

In [40]:
# Now, we can handle the categorical values, "Country" and "Status"




COUNTRY:  Afghanistan 

[]
COUNTRY:  Albania 

[]
COUNTRY:  Algeria 

['Alcohol', 'Hepatitis B', 'Total expenditure']
COUNTRY:  Angola 

['Alcohol', 'Hepatitis B', 'Total expenditure']
COUNTRY:  Antigua and Barbuda 

['Alcohol', 'Hepatitis B', 'Total expenditure', 'Population']
COUNTRY:  Argentina 

['Alcohol', 'Hepatitis B', 'Total expenditure']
COUNTRY:  Armenia 

['Alcohol', 'Total expenditure']
COUNTRY:  Australia 

['Alcohol', 'Hepatitis B', 'Total expenditure']
COUNTRY:  Austria 

['Alcohol', 'Total expenditure']
COUNTRY:  Azerbaijan 

['Alcohol', 'Hepatitis B', 'Total expenditure']
COUNTRY:  Bahamas 

['Alcohol', 'Hepatitis B', 'Total expenditure', 'GDP', 'Population']
COUNTRY:  Bahrain 

['Alcohol', 'Total expenditure', 'Population']
COUNTRY:  Bangladesh 

['Alcohol', 'Hepatitis B', 'Total expenditure']
COUNTRY:  Barbados 

['Alcohol', 'Hepatitis B', 'Total expenditure', 'Population']
COUNTRY:  Belarus 

['Alcohol', 'Total expenditure']
COUNTRY:  Belgium 

['Alcohol', 'Total ex